In [58]:
import torch
from einops import rearrange


def horizontal_forward_scan(input_tensor):
    """
    对输入张量进行水平正向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    # 将张量展平为 (B * H, W, C) 形状
    flattened = input_tensor.reshape(-1, W, C)
    return flattened.view(B, H * W, C).permute(0, 2, 1)


def horizontal_backward_scan(input_tensor):
    """
    对输入张量进行水平反向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    # 对最后一维（水平方向）进行翻转
    reversed_tensor = torch.flip(input_tensor, dims=[1])
    # 将张量展平为 (B * H, W, C) 形状
    flattened = reversed_tensor.reshape(-1, W, C)
    return flattened.view(B, H * W, C).permute(0, 2, 1)

def horizontal_backward_scan_flipfirst(input_tensor):
    """
    对输入张量进行水平反向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = torch.flip(input_tensor, dims=[-1])
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    # 对最后一维（水平方向）进行翻转
    # reversed_tensor = torch.flip(input_tensor, dims=[1])
    # 将张量展平为 (B * H, W, C) 形状
    flattened = input_tensor.reshape(-1, W, C)
    return flattened.view(B, H * W, C).permute(0, 2, 1)


def vertical_forward_scan(input_tensor):
    """
    对输入张量进行垂直正向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对倒数第二维（垂直方向）进行操作
    input_tensor = input_tensor.permute(0, 1, 3, 2)
    print(input_tensor)
    print(input_tensor.shape)
    # 将张量转置为 (B * W, H, C) 形状
    # transposed = rearrange(input_tensor, 'B H W C -> (B W) H C')
    # return transposed.view(B, W * H, C).permute(0, 2, 1)
    return input_tensor.flatten(2).permute(0,2,1)


def vertical_backward_scan(input_tensor):
    """
    对输入张量进行垂直反向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对倒数第二维（垂直方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    # 对倒数第二维（垂直方向）进行翻转
    reversed_tensor = torch.flip(input_tensor, dims=[2])
    # 将张量转置为 (B * W, H, C) 形状
    transposed = rearrange(reversed_tensor, 'B H W C -> (B W) H C')
    return transposed.view(B, W * H, C).permute(0, 2, 1)

def vertical_backward_scan(input_tensor):
    """
    对输入张量进行垂直反向扫描
    """
    B, C, H, W = input_tensor.shape
    input_tensor = torch.flip(input_tensor, dims=[-2]).contiguous()
    
    input_tensor = input_tensor.permute(0, 2, 3, 1)

    # 将张量转置为 (B * W, H, C) 形状
    transposed = rearrange(input_tensor, 'B H W C -> (B W) H C').contiguous()
    return transposed.view(B, W * H, C).permute(0, 2, 1)


def in_horizontal_scan(input_tensor):
    """
    对输入张量进行水平向内扫描
    """
    B, C, H, W = input_tensor.shape
    mid = W // 2
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    left_half = input_tensor[:, :, :mid, :]
    right_half = torch.flip(input_tensor[:, :, mid:, :], dims=[2])
    # 拼接左右两部分
    print("left", left_half.flatten())
    print("right", right_half.flatten())
    
    combined = torch.cat([left_half, right_half], dim=2).contiguous()
    # 将张量展平为 (B * H, W, C) 形状
    flattened = combined.reshape(-1, W, C).contiguous()
    return flattened.view(B, H * W, C).permute(0, 2, 1)


def out_horizontal_scan(input_tensor):
    """
    对输入张量进行水平向外扫描
    """
    B, C, H, W = input_tensor.shape
    mid = W // 2
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    left_half = torch.flip(input_tensor[:, :, :mid, :], dims=[2])
    right_half = input_tensor[:, :, mid:, :]
    # 拼接左右两部分
    print("left", left_half.flatten())
    print("right", right_half.flatten())    
    combined = torch.cat([left_half, right_half], dim=2).contiguous()
    # 将张量展平为 (B * H, W, C) 形状
    flattened = combined.reshape(-1, W, C).contiguous()
    return flattened.view(B, H * W, C).permute(0, 2, 1)


def in_vertical_scan(input_tensor):
    """
    对输入张量进行垂直向内扫描
    """
    B, C, H, W = input_tensor.shape
    mid = H // 2
    # 将 NCHW 张量转置为 NHWC 格式，以便对倒数第二维（垂直方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    top_half = input_tensor[:, :mid, :, :]
    bottom_half = torch.flip(input_tensor[:, mid:, :, :], dims=[1])
    # 拼接上下两部分
    print("top_half",top_half.flatten())
    print("bottom", bottom_half.flatten())    
    combined = torch.cat([top_half, bottom_half], dim=1)
    # 将张量转置为 (B * W, H, C) 形状
    transposed = rearrange(combined, 'B H W C -> (B W) H C').contiguous()
    return transposed.view(B, W * H, C).permute(0, 2, 1)


def out_vertical_scan(input_tensor):
    """
    对输入张量进行垂直向外扫描
    """
    B, C, H, W = input_tensor.shape
    mid = H // 2
    # 将 NCHW 张量转置为 NHWC 格式，以便对倒数第二维（垂直方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    top_half = torch.flip(input_tensor[:, :mid, :, :], dims=[1])
    bottom_half = input_tensor[:, mid:, :, :]
    # 拼接上下两部分
    # 拼接上下两部分
    print("top_half",top_half.flatten())
    print("bottom", bottom_half.flatten())       
    combined = torch.cat([top_half, bottom_half], dim=1).contiguous()
    # 将张量转置为 (B * W, H, C) 形状
    transposed = rearrange(combined, 'B H W C -> (B W) H C').contiguous()
    return transposed.view(B, W * H, C).permute(0, 2, 1)


In [1]:
import torch

def vertical_forward_scan(input_tensor):
    """
    对输入张量进行垂直正向扫描
    输入形状: (B, C, H, W)
    变换后形状: (B, W * H, C)
    """
    B, C, H, W = input_tensor.shape
    # 交换 H 和 W 维度
    input_tensor = input_tensor.permute(0, 1, 3, 2)  # 变为 (B, C, W, H)
    return input_tensor.flatten(2).permute(0, 2, 1)  # 变为 (B, W*H, C)


def vertical_forward_scan_inv(transformed_tensor, original_shape):
    """
    逆变换: 复原 vertical_forward_scan 变换后的数据
    输入形状: (B, W * H, C)
    复原形状: (B, C, H, W)
    """
    B, C, H, W = original_shape
    # 先 permute 回 (B, C, W*H)
    transformed_tensor = transformed_tensor.permute(0, 2, 1)  # (B, C, W*H)
    # 复原为 (B, C, W, H)
    recovered = transformed_tensor.view(B, C, W, H)
    # 再交换 H 和 W 维度
    return recovered.permute(0, 1, 3, 2)  # (B, C, H, W)


# 测试代码
B, C, H, W = 1, 1, 3, 3  # 形状定义
input_tensor = torch.tensor([[[[1, 2, 3], 
                               [4, 5, 6], 
                               [7, 8, 9]]]], dtype=torch.float32)  # (1,1,3,3)

# 进行 forward 变换
transformed = vertical_forward_scan(input_tensor)
print("Transformed Tensor:\n", transformed)

# 进行 inverse 逆变换
recovered = vertical_forward_scan_inv(transformed, input_tensor.shape)
print("Recovered Tensor:\n", recovered)

# 验证恢复是否正确
assert torch.allclose(input_tensor, recovered), "Inverse transform failed!"


Transformed Tensor:
 tensor([[[1.],
         [4.],
         [7.],
         [2.],
         [5.],
         [8.],
         [3.],
         [6.],
         [9.]]])
Recovered Tensor:
 tensor([[[[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]])


In [6]:
from einops import rearrange

def horizontal_forward_scan(input_tensor):
    """
    对输入张量进行水平正向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    # 将张量展平为 (B * H, W, C) 形状
    flattened = input_tensor.reshape(-1, W, C)
    return flattened.view(B, H * W, C).permute(0, 2, 1)


def horizontal_backward_scan(input_tensor):
    """
    对输入张量进行水平反向扫描
    """
    B, C, H, W = input_tensor.shape
    # 将 NCHW 张量转置为 NHWC 格式，以便对最后一维（水平方向）进行操作
    input_tensor = torch.flip(input_tensor, dims=[-1])
    input_tensor = input_tensor.permute(0, 2, 3, 1)
    # 对最后一维（水平方向）进行翻转
    # reversed_tensor = torch.flip(input_tensor, dims=[1])
    # 将张量展平为 (B * H, W, C) 形状
    flattened = input_tensor.reshape(-1, W, C)
    return flattened.view(B, H * W, C).permute(0, 2, 1)

def vertical_backward_scan(input_tensor):
    """
    对输入张量进行垂直反向扫描
    """
    B, C, H, W = input_tensor.shape
    input_tensor = torch.flip(input_tensor, dims=[-2]).contiguous()
    
    input_tensor = input_tensor.permute(0, 2, 3, 1)

    # 将张量转置为 (B * W, H, C) 形状
    transposed = rearrange(input_tensor, 'B H W C -> (B W) H C').contiguous()
    return transposed.view(B, W * H, C).permute(0, 2, 1)

print(horizontal_backward_scan(input_tensor))

horizontal_backward_scan_flipfirst(input_tensor)


tensor([[[7., 8., 9., 4., 5., 6., 1., 2., 3.]]])


tensor([[[3., 2., 1., 6., 5., 4., 9., 8., 7.]]])

In [7]:
import torch
from einops import rearrange

def horizontal_forward_scan(input_tensor):
    """
    对输入张量进行水平正向扫描
    输入形状: (B, C, H, W)
    变换后形状: (B, H * W, C)
    """
    B, C, H, W = input_tensor.shape
    input_tensor = input_tensor.permute(0, 2, 3, 1)  # (B, H, W, C)
    flattened = input_tensor.reshape(B, H * W, C)  # (B, H * W, C)
    return flattened.permute(0, 2, 1)  # (B, C, H * W)

def horizontal_forward_scan_inv(transformed_tensor, original_shape):
    """
    逆变换: 复原 horizontal_forward_scan 变换后的数据
    输入形状: (B, C, H * W)
    复原形状: (B, C, H, W)
    """
    B, C, H, W = original_shape
    transformed_tensor = transformed_tensor.permute(0, 2, 1)  # (B, H * W, C)
    recovered = transformed_tensor.view(B, H, W, C)  # 复原为 (B, H, W, C)
    return recovered.permute(0, 3, 1, 2)  # (B, C, H, W)


def horizontal_backward_scan(input_tensor):
    """
    对输入张量进行水平反向扫描
    """
    B, C, H, W = input_tensor.shape
    input_tensor = torch.flip(input_tensor, dims=[-1])  # 水平翻转 (B, C, H, W)
    input_tensor = input_tensor.permute(0, 2, 3, 1)  # (B, H, W, C)
    flattened = input_tensor.reshape(B, H * W, C)  # (B, H * W, C)
    return flattened.permute(0, 2, 1)  # (B, C, H * W)

def horizontal_backward_scan_inv(transformed_tensor, original_shape):
    """
    逆变换: 复原 horizontal_backward_scan 变换后的数据
    """
    B, C, H, W = original_shape
    transformed_tensor = transformed_tensor.permute(0, 2, 1)  # (B, H * W, C)
    recovered = transformed_tensor.view(B, H, W, C)  # (B, H, W, C)
    recovered = recovered.permute(0, 3, 1, 2)  # (B, C, H, W)
    return torch.flip(recovered, dims=[-1])  # 水平翻转回去


def vertical_forward_scan(input_tensor):
    """
    对输入张量进行垂直正向扫描
    """
    B, C, H, W = input_tensor.shape
    input_tensor = input_tensor.permute(0, 1, 3, 2)  # (B, C, W, H)
    return input_tensor.flatten(2).permute(0, 2, 1)  # (B, W*H, C)

def vertical_forward_scan_inv(transformed_tensor, original_shape):
    """
    逆变换: 复原 vertical_forward_scan 变换后的数据
    """
    B, C, H, W = original_shape
    transformed_tensor = transformed_tensor.permute(0, 2, 1)  # (B, C, W*H)
    recovered = transformed_tensor.view(B, C, W, H)  # (B, C, W, H)
    return recovered.permute(0, 1, 3, 2)  # (B, C, H, W)


def vertical_backward_scan(input_tensor):
    """
    对输入张量进行垂直反向扫描
    """
    B, C, H, W = input_tensor.shape
    input_tensor = torch.flip(input_tensor, dims=[-2]).contiguous()  # 垂直翻转
    input_tensor = input_tensor.permute(0, 1, 3, 2)  # (B, C, W, H)
    return input_tensor.flatten(2).permute(0, 2, 1)  # (B, W*H, C)

def vertical_backward_scan_inv(transformed_tensor, original_shape):
    """
    逆变换: 复原 vertical_backward_scan 变换后的数据
    """
    B, C, H, W = original_shape
    transformed_tensor = transformed_tensor.permute(0, 2, 1)  # (B, C, W*H)
    recovered = transformed_tensor.view(B, C, W, H)  # (B, C, W, H)
    recovered = recovered.permute(0, 1, 3, 2)  # (B, C, H, W)
    return torch.flip(recovered, dims=[-2])  # 垂直翻转回去


# 测试代码
B, C, H, W = 1, 1, 3, 3  # 形状定义
input_tensor = torch.tensor([[[[1, 2, 3], 
                               [4, 5, 6], 
                               [7, 8, 9]]]], dtype=torch.float32)  # (1,1,3,3)

# 测试 horizontal_forward_scan
transformed = horizontal_forward_scan(input_tensor)
recovered = horizontal_forward_scan_inv(transformed, input_tensor.shape)
assert torch.allclose(input_tensor, recovered), "horizontal_forward_scan_inv failed!"

# 测试 horizontal_backward_scan
transformed = horizontal_backward_scan(input_tensor)
recovered = horizontal_backward_scan_inv(transformed, input_tensor.shape)
assert torch.allclose(input_tensor, recovered), "horizontal_backward_scan_inv failed!"

# 测试 vertical_forward_scan
transformed = vertical_forward_scan(input_tensor)
recovered = vertical_forward_scan_inv(transformed, input_tensor.shape)
assert torch.allclose(input_tensor, recovered), "vertical_forward_scan_inv failed!"

# 测试 vertical_backward_scan
transformed = vertical_backward_scan(input_tensor)
recovered = vertical_backward_scan_inv(transformed, input_tensor.shape)
assert torch.allclose(input_tensor, recovered), "vertical_backward_scan_inv failed!"

print("所有变换及其逆变换测试通过！")


所有变换及其逆变换测试通过！


In [ ]:
def octant_shift_grouped(x, groups=8):
    """
    对 4D 张量 (NCHW) 进行分组八面移位操作。
    :param x: 输入张量，形状为 (N, C, H, W)
    :param groups: 分组数，默认为 8
    :return: 位移后的特征图，形状为 (N, C * 8, H, W)
    """
    N, C, H, W = x.shape
    assert C % groups == 0, "通道数必须能被分组数整除"
    channels_per_group = C // groups

    # 将输入张量按通道分组
    x_grouped = x.view(N, groups, channels_per_group, H, W)

    # 定义 8 种位移方向
    shifts = [
        (0, 0),    # 不位移
        (0, 1),    # 宽度方向右移
        (0, -1),   # 宽度方向左移
        (1, 0),    # 高度方向下移
        (-1, 0),   # 高度方向上移
        (1, 1),    # 右下方向位移
        (1, -1),   # 左下方向位移
        (-1, 1),   # 右上方向位移
        (-1, -1),  # 左上方向位移
    ]

    # 对每一组进行八面移位
    shifted_features = []
    for h_shift, w_shift in shifts:
        
        shifted_x = torch.roll(x_grouped, shifts=(h_shift, w_shift), dims=(-2, -1))
        shifted_features.append(shifted_x)

    # 将位移结果按通道维度拼接
    shifted_features = torch.cat(shifted_features, dim=2)  # (N, groups, C * 8, H, W)
    shifted_features = shifted_features.view(N, -1, H, W)  # (N, C * 8, H, W)

    return shifted_features


In [61]:
import torch.nn as nn
class GSC(nn.Module):
    def __init__(self, in_channels):
        super(GSC, self).__init__()
        self.proj3x3 = nn.Conv2d(in_channels, 4*in_channels, kernel_size=3, padding=1)
        self.proj1x1 = nn.Conv2d(in_channels, in_channels, kernel_size=1, padding=0)
        
        
        self.norm = nn.InstanceNorm2d(in_channels)
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()

        

    def forward(self, x):
        x_residual = x

        x1 = self.norm(self.bn(self.proj3x3(x)))
        x1g = self.norm(self.bn(self.proj1x1(x)))
        
        x1 = x1*x1g
        
        x2= self.norm(self.bn(self.proj3x3(x1)))
        
        return x2 + x_residual


In [ ]:
gsc = GSC(in_channels=1)
from thop import 


In [62]:
a = torch.tensor([[[[1, 2, 3,4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]]], dtype=torch.float32)
gsc = GSC(in_channels=1)
gsc(a).shape


torch.Size([1, 1, 4, 4])

In [27]:
a = torch.tensor([[[[1, 2, 3,4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]]], dtype=torch.float32)
a


tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [37]:
horizontal_backward_scan_flipfirst(a)


tensor([[[ 4.,  3.,  2.,  1.,  8.,  7.,  6.,  5., 12., 11., 10.,  9., 16., 15.,
          14., 13.]]])

In [39]:
vertical_forward_scan(a)


tensor([[[[ 1.,  5.,  9., 13.],
          [ 2.,  6., 10., 14.],
          [ 3.,  7., 11., 15.],
          [ 4.,  8., 12., 16.]]]])
torch.Size([1, 1, 4, 4])


tensor([[[ 1.],
         [ 5.],
         [ 9.],
         [13.],
         [ 2.],
         [ 6.],
         [10.],
         [14.],
         [ 3.],
         [ 7.],
         [11.],
         [15.],
         [ 4.],
         [ 8.],
         [12.],
         [16.]]])

In [42]:
vertical_backward_scan(a)


tensor([[[13.,  9.,  5.,  1., 14., 10.,  6.,  2., 15., 11.,  7.,  3., 16., 12.,
           8.,  4.]]])

In [48]:
in_horizontal_scan(a)


half tensor([ 1.,  2.,  5.,  6.,  9., 10., 13., 14.])
right tensor([[[[ 4.],
          [ 3.]],

         [[ 8.],
          [ 7.]],

         [[12.],
          [11.]],

         [[16.],
          [15.]]]])


tensor([[[ 1.,  2.,  4.,  3.,  5.,  6.,  8.,  7.,  9., 10., 12., 11., 13., 14.,
          16., 15.]]])

In [50]:
out_horizontal_scan(a)


left tensor([ 2.,  1.,  6.,  5., 10.,  9., 14., 13.])
right tensor([ 3.,  4.,  7.,  8., 11., 12., 15., 16.])


tensor([[[ 2.,  1.,  3.,  4.,  6.,  5.,  7.,  8., 10.,  9., 11., 12., 14., 13.,
          15., 16.]]])

In [57]:
in_vertical_scan(a)


top_half tensor([1., 2., 3., 4., 5., 6., 7., 8.])
bottom tensor([13., 14., 15., 16.,  9., 10., 11., 12.])


tensor([[[ 1.,  5., 13.,  9.,  2.,  6., 14., 10.,  3.,  7., 15., 11.,  4.,  8.,
          16., 12.]]])

In [53]:
out_vertical_scan(a)


top_half tensor([5., 6., 7., 8., 1., 2., 3., 4.])
bottom tensor([ 9., 10., 11., 12., 13., 14., 15., 16.])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [55]:
out_vertical_scan(a)


top_half tensor([5., 6., 7., 8., 1., 2., 3., 4.])
bottom tensor([ 9., 10., 11., 12., 13., 14., 15., 16.])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [54]:
in_vertical_scan(a)


top_half tensor([1., 2., 3., 4., 5., 6., 7., 8.])
bottom tensor([13., 14., 15., 16.,  9., 10., 11., 12.])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [29]:
vertical_forward_scan(a)


tensor([[[[ 1.,  5.,  9., 13.],
          [ 2.,  6., 10., 14.],
          [ 3.,  7., 11., 15.],
          [ 4.,  8., 12., 16.]]]])
torch.Size([1, 1, 4, 4])


tensor([[[ 1.],
         [ 5.],
         [ 9.],
         [13.],
         [ 2.],
         [ 6.],
         [10.],
         [14.],
         [ 3.],
         [ 7.],
         [11.],
         [15.],
         [ 4.],
         [ 8.],
         [12.],
         [16.]]])

In [21]:
a.flatten(2)


tensor([[[ 1.3081, -1.7051, -0.6799, -0.2679, -0.1162,  1.6008,  1.6356,
          -0.6943,  1.2421, -0.3325,  0.1871,  0.9646,  0.9783, -0.3929,
          -0.5501,  1.2272]]])

In [23]:
a.flatten(2).permute(0,2,1)


tensor([[[ 1.3081],
         [-1.7051],
         [-0.6799],
         [-0.2679],
         [-0.1162],
         [ 1.6008],
         [ 1.6356],
         [-0.6943],
         [ 1.2421],
         [-0.3325],
         [ 0.1871],
         [ 0.9646],
         [ 0.9783],
         [-0.3929],
         [-0.5501],
         [ 1.2272]]])

In [11]:
input_tensor = torch.tensor([[[[1, 2, 3,4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]]], dtype=torch.float32)
print("Horizontal Forward Scan:")
print(vertical_forward_scan(input_tensor))


Horizontal Forward Scan:
tensor([[[[ 1.,  5.,  9., 13.],
          [ 2.,  6., 10., 14.],
          [ 3.,  7., 11., 15.],
          [ 4.,  8., 12., 16.]]]])
torch.Size([1, 1, 4, 4])
tensor([[[ 1.],
         [ 5.],
         [ 9.],
         [13.],
         [ 2.],
         [ 6.],
         [10.],
         [14.],
         [ 3.],
         [ 7.],
         [11.],
         [15.],
         [ 4.],
         [ 8.],
         [12.],
         [16.]]])


In [2]:


# 示例使用
input_tensor = torch.tensor([[[[1, 2, 3], [4, 5, 6], [7, 8, 9]]]], dtype=torch.float32)

print("Horizontal Forward Scan:")
print(horizontal_forward_scan(input_tensor))
print("Horizontal Backward Scan:")
print(horizontal_backward_scan(input_tensor))
print("Vertical Forward Scan:")
print(vertical_forward_scan(input_tensor))
print("Vertical Backward Scan:")
print(vertical_backward_scan(input_tensor))
print("In Horizontal Scan:")
print(in_horizontal_scan(input_tensor))
print("Out Horizontal Scan:")
print(out_horizontal_scan(input_tensor))
print("In Vertical Scan:")
print(in_vertical_scan(input_tensor))
print("Out Vertical Scan:")
print(out_vertical_scan(input_tensor))


Horizontal Forward Scan:
tensor([[[1., 2., 3., 4., 5., 6., 7., 8., 9.]]])
Horizontal Backward Scan:
tensor([[[7., 8., 9., 4., 5., 6., 1., 2., 3.]]])
Vertical Forward Scan:


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.